In [ ]:
from functions import *
import itertools
from matplotlib import pyplot as plt
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from tqdm import trange

%load_ext autoreload
%autoreload 2
plt.rcParams.update({'font.size': 20})

In [ ]:
## ----- use only some classes or all the data
folder=folder_bin= DATA_PATH

# Find the parameters (as in find_ising.py)

In [ ]:
d = 500
label='all'
# label=1

n_samples = 10
n_thermalize = 50000
adam_step = 1e-2
n_step_thermalize = 50
n_step_compute_f = 20
n_step_decorrelate = 2
n_steps = 10000
adam_step=1e-4
eps=1e-7

errs = np.loadtxt(f'{folder}f_dataERR_{d}_{label}.dat')
# errs = np.ones(len(f_data))*1e-4

In [ ]:
# --------------------- LARGE SYSTEMS FROM HERE _______________________________________
f_data = np.loadtxt(f'{folder}f_data_{d}_{label}.dat')
q1 = np.log((1-f_data[:d]+eps)/(1+f_data[:d]+eps))/2*d
q1 = np.hstack((q1, [0]*int(d*(d-1)/2)))
if not os.path.isfile(f'{folder}q_{d}_{label}.dat'):
    print('no q found, start from independent')
#     q = np.zeros(len(f_data))
    q1 = np.log((1-f_data[:d]+eps)/(1+f_data[:d]+eps))/2*d
    q = np.hstack((q1, [0]*int(d*(d-1)/2)))
    q= tf.Variable(q, dtype=tf.float64)
    
else:
    print('q loaded from file')
    q = np.loadtxt(f'{folder}q_{d}_{label}.dat')
#     q = np.zeros(len(f_data))
#     q1 = np.log((1-f_data[:d])/(1+f_data[:d]))/2*d
#     q = np.hstack((q1, [0]*int(d*(d-1)/2)))

q= tf.Variable(q, dtype=tf.float64)
h,J = get_hJ(q,d)
h = tf.convert_to_tensor(h,dtype=tf.float64)
J= tf.convert_to_tensor(J,dtype=tf.float64)
if d==500: n_samples = 8000
else: n_samples = 20000
x = np.random.choice([-1,1], (n_samples,d))
x = tf.convert_to_tensor(x,dtype=tf.float64) 

In [ ]:
h1, J1 = get_hJ(q1,d)

In [ ]:
# zsum = np.loadtxt(f'{folder}_histosum_{d}_{label}.dat')
# zsums = np.loadtxt(f'{folder}_histosums_{d}_{label}.dat')
pzsum,zsum = np.loadtxt(f'{folder}histosum_{d}_{label}.dat')

In [ ]:
n_thermalize = 200000
x, energy_history = thermalize(x,J,h, NumSteps=n_thermalize , crange =trange)
plt.plot(energy_history)

In [ ]:
x, f_model = compute_f(x,J,h,StepsTherm=200,NumSteps=300, crange=lambda NumSteps, dtype: trange(NumSteps))

In [ ]:
f_data = tf.convert_to_tensor(f_data, tf.float64)

plt.scatter(f_data[:d].numpy(),f_model[:d].numpy(),s=5)
plt.scatter(f_data[d:].numpy(),f_model[d:].numpy(),s=5)
plt.scatter(f_data[sorted_different_indeces[0]].numpy(),f_model[sorted_different_indeces[0]].numpy(),s=10)
plt.scatter(f_data[sorted_different_indeces[1]].numpy(),f_model[sorted_different_indeces[1]].numpy(),s=10)

xmin, xmax = plt.gca().get_xlim()
xx = np.linspace(xmin,xmax,100)
plt.plot(xx,xx, color='red')

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.scatter(errs, abs(f_data.numpy()-f_model.numpy()), s=0.1)
xmin,xmax=plt.gca().get_xlim()
xx=np.linspace(xmin,xmax,100)
plt.plot(xx,xx, color='red')
plt.plot(xx,xx*3, color='red')
plt.plot(xx,xx*5, color='red', ls='--')

plt.subplot(1,2,2)
plt.plot(abs(f_data.numpy()-f_model.numpy()))
plt.plot(errs)
plt.plot(errs*3)

plt.title((abs(f_data.numpy()-f_model.numpy()) > 3*errs).sum())
plt.tight_layout()

In [ ]:
np.savetxt(f'{folder}f_model_{d}_{label}.dat',f_model.numpy())

In [ ]:
### ----- plot sum
s=[]
for _ in trange(100):
    x, energy_history = thermalize(x,J,h, NumSteps=50 , crange =range)
    s.append(x.numpy().sum(1))
s=np.array(s).flatten()
h0,h1 = np.unique(s, return_counts=True); h1=h1/h1.sum()
plt.plot(h0,h1, '-', label='model', color='red')
plt.plot(zsum,pzsum, '-', label='data', color='black')
plt.xlabel('sum')
plt.ylabel('P(sum)')

## EVOLUTION

In [ ]:
d,label

In [ ]:
f_model_history =[]
q_history=[]
mre_history=[]
m2conn_diff =[]

In [ ]:
f_data = tf.convert_to_tensor(f_data, tf.float64)

In [ ]:
sorted_different_indeces = np.argsort(abs((np.array(f_data - f_model)))/errs)[::-1]
sorted_different_indeces = np.argsort(abs((np.array(f_data - f_model))))[::-1]

i = sorted_different_indeces[:10]
# print(i,(abs((np.array(f_data - f_model)))/errs)[i])
# print(i,(abs((np.array(f_data - f_model))))[i])

In [ ]:
# i=[100]
d, label

In [ ]:
# (abs((np.array(f_data - f_model)))/errs).argmax()
np.savetxt(f'{folder}q_{d}_{label}.dat', np.array(q))

In [ ]:
# ---------------------- THERMALIZE
adam_step = 1e-4
optimizer = tf.optimizers.Adam(adam_step)

n_steps =500
n_step_thermalize = 200
n_step_compute_f = 200
n_step_decorrelate = 200

for t in trange(n_steps):
    h,J = get_hJ(q,d)
    h = tf.convert_to_tensor(h,dtype=tf.float64)
    J= tf.convert_to_tensor(J,dtype=tf.float64)
    x,f_model = step(q,x,J,h,f_data, n_step_thermalize, n_step_compute_f,n_step_decorrelate, optimizer)
    
    f_model_history.append(f_model.numpy()[i])
    q_history.append(q.numpy()[i]) 
       
    mre_history.append(tf.reduce_mean((f_model-f_data)**2).numpy())
    np.savetxt(f'{folder}q_{d}_{label}.dat', np.array(q))
    
    ## -------------------------- PLOTS -----------------------
    ## --------------------------------------------------------
    if ((t)%int(n_steps/20)==0):
        x, energy_history = thermalize(x,J,h, NumSteps=1000 , crange =range)
        #__________________________________________________________________________________
        plt.figure(figsize=(15,10))
        plt.subplot(2,3,1)
        plt.plot(mre_history[:])
        plt.xlabel('epochs')
        plt.ylabel(r'||$f_{data} - f_{model}||^2$')
        # 
        # ---------------- Plot parameters evolution ------------------
        
        plt.subplot(2,3,2)
        for isel in range(5):
            plt.plot((np.array(f_model_history)[:,isel] - np.array(f_data)[i[isel]])/errs[i[isel]])
            plt.axhline(0)
            plt.axhline(3)
            plt.axhline(-3)
            plt.title(f'Abs diff {(np.array(f_model_history)[-1,isel] - np.array(f_data)[i[isel]]):.2f}')
            
            
        plt.xlabel('epochs')
        plt.ylabel(r'$f_{model}/f_{data}$')
        # ---------
        plt.subplot(2,3,3)
        plt.plot(np.array(q_history)[:,:2]/np.array(q_history)[0,:2])
#         plt.plot(np.array(q_history)[:,])
        
        plt.title(f'Parameters {i[0]}')
        plt.xlabel('epochs')
        plt.ylabel('q')
        plt.tight_layout()
          
        # ---------
        plt.subplot(2,3,4)
        plt.plot(abs(f_data.numpy()-f_model.numpy())/errs)
#         plt.plot(errs)
#         plt.plot(errs*3)        
        plt.axhline(1)
        plt.axhline(3)
        plt.title(f'{(abs(f_data.numpy()-f_model.numpy())/errs).max():.4f}')

        plt.subplot(2,3,5)
        m2conn_data = np.array(f_data[d:] - np.array(f_data[:d][:,None]*f_data[:d][None,:])[np.triu_indices(d,1)])
        m2conn_model = np.array(f_model[d:]-np.array(f_model[:d][:,None]*f_model[:d][None,:])[np.triu_indices(d,1)])
        plt.scatter(m2conn_data[::10] ,m2conn_model[::10], s=5)
        xmin, xmax = plt.gca().get_xlim()
        xx = np.linspace(xmin,xmax,100)
        plt.plot(xx,xx, color='red')
        plt.title(r'$\langle x_i x_j\rangle_{conn}$')
        
        plt.subplot(2,3,6)
        s=[]
        for _ in range(10):
            x, energy_history = thermalize(x,J,h, NumSteps=20 , crange =range)
            s.append(x.numpy().sum(1))
        s=np.array(s).flatten()
        h0,h1 = np.unique(s, return_counts=True); h1=h1/h1.sum()
        plt.plot(h0,h1, '-', label='model', color='red')
        plt.plot(zsum,pzsum, '-', label='data', color='black')
        plt.xlabel('sum')
        plt.ylabel('P(sum)')
        plt.ylim(0,0.015)
  
        plt.tight_layout()
        plt.savefig(f"{folder}evolution_{d}_{label}.pdf", format="pdf", bbox_inches="tight")
        plt.show()
        
    if ((abs(f_data.numpy()-f_model.numpy())<3*errs).sum()==0): break

In [ ]:
# q_saved=np.copy(q)

In [ ]:
for ii in range(len(i)):
    plt.subplot(1,2,1)
    plt.plot(np.array(q_history)[:,ii])
    plt.subplot(1,2,2)
    plt.plot(np.array(f_model_history)[:,ii])
    plt.axhline(np.array(f_data)[i[ii]])
    plt.axhline(np.array(f_data)[i[ii]] + 3*errs[i[ii]])
    plt.axhline(np.array(f_data)[i[ii]] -3*errs[i[ii]])
    plt.tight_layout()
    plt.show()

In [ ]:
# fmh = np.copy(f_model_history)
# plt.plot(np.array(fmh))
# plt.plot(np.array(f_model_history))

# plt.axhline(np.array(f_data)[i])
# plt.axhline(3*errs[i]+np.array(f_data)[i])
# plt.axhline(-3*errs[i]+np.array(f_data)[i])

In [ ]:
n_thermalize = 50000
x, energy_history = thermalize(x,J,h, NumSteps=n_thermalize , crange =trange)
plt.plot(energy_history)

In [ ]:
x, f_model = compute_f(x,J,h,StepsTherm=200,NumSteps=300, crange=lambda NumSteps, dtype: trange(NumSteps))

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.scatter(errs, abs(f_data.numpy()-f_model.numpy()))
xmin,xmax=plt.gca().get_xlim()
xx=np.linspace(xmin,xmax,100)
plt.plot(xx,xx, color='red')
plt.plot(xx,xx*3, color='red')
plt.plot(xx,xx*5, color='red')

plt.subplot(1,2,2)
plt.plot(abs(f_data.numpy()-f_model.numpy()))
plt.plot(errs)
plt.plot(errs*3)
# plt.title((abs(f_data.numpy()-f_model.numpy()) > 3*errs).sum())
plt.title(f'{(abs(f_data.numpy()-f_model.numpy())/errs).max():.2f}')

plt.tight_layout()

In [ ]:
# ---------- paramters
plt.subplot(1,2,1)
plt.scatter(f_data.numpy()[:d], q.numpy()[:d]/d, s=1)

plt.subplot(1,2,2)
m2conn_data = np.array(f_data[d:] - np.array(f_data[:d][:,None]*f_data[:d][None,:])[np.triu_indices(d,1)])
# m2conn_model = np.array(f_model[d:]-np.array(f_model[:d][:,None]*f_model[:d][None,:])[np.triu_indices(d,1)])
plt.scatter(m2conn_data, q.numpy()[d:]/d, s=0.05, alpha=0.5)

plt.tight_layout()

In [ ]:
np.savetxt(f'{folder}q_copy_{d}_{label}.dat', np.array(q))
# 

In [ ]:
# q = np.loadtxt(f'{folder}q_copy_{d}_{label}.dat')


## moments

In [ ]:
# n_samples = 8000
# x = np.random.choice([-1,1], (n_samples,d))
# x = tf.convert_to_tensor(x,dtype=tf.float64) 

In [ ]:
# n_thermalize = 20000
# x, energy_history = thermalize(x,J,h, NumSteps=n_thermalize , crange =trange)
# plt.plot(energy_history)

In [ ]:
m2 = np.loadtxt(f'{folder}_m2_{d}_{label}.dat')[:,0]
m3 = np.loadtxt(f'{folder}_m3_{d}_{label}.dat')[:,0]
m4 = np.loadtxt(f'{folder}_m4_{d}_{label}.dat')[:,0]
indices = np.loadtxt(f'{folder}_m4_{d}_{label}.dat')[:,1:].astype(int)

m2_model=np.zeros(len(m2))
m3_model=np.zeros(len(m2))
m4_model=np.zeros(len(m2))

In [ ]:
ntries=500
for _ in trange(ntries):
    x, energy_history = thermalize(x,J,h, NumSteps=1000 , crange =range)
    dx = x.numpy()-x.numpy().mean(0)
    for ii in range(len(indices)):
        i,j,k,l = indices[ii]   
        m2_model[ii]+= (dx[:, [i,j]].prod(1).mean())
        m3_model[ii]+= (dx[:, [i,j,k]].prod(1).mean())
        m4_model[ii]+= (dx[:, [i,j,k,l]].prod(1).mean())
m2_model/=ntries
m3_model/=ntries
m4_model/=ntries

np.savetxt(f'{folder}_m2model_{d}_all.dat',m2_model)
np.savetxt(f'{folder}_m3model_{d}_all.dat',m3_model)
np.savetxt(f'{folder}_m4model_{d}_all.dat',m4_model)

In [ ]:
# plt.scatter(m2,m2_model, s=2)
plt.scatter(m3,m3_model, s=2)
plt.scatter(m4,m4_model, s=2)

xmin, xmax = plt.gca().get_xlim()
xx = np.linspace(xmin,xmax,100)
plt.plot(xx,xx, color='red')

In [ ]:
# # ---------- higher moments IN CLASS
# if label!='all':
#     folder = '../../../../scratch/network/csarra/allen_saved/binary/'
#     ijkl = (np.loadtxt(f'{folder}_m4_{d}_{label}.dat')[:,1:]).astype(int)

#     m2 = np.loadtxt(f'{folder}_m2_{d}_{label}.dat')[:,0]
#     m3 = np.loadtxt(f'{folder}_m3_{d}_{label}.dat')[:,0]
#     m4 = np.loadtxt(f'{folder}_m4_{d}_{label}.dat')[:,0]

#     dx= np.array(x-f_model[:d])

#     moms =[]
#     for i,j,k,l in ijkl:
#         moms.append(((dx[:,i]*dx[:,j]).mean(),(dx[:,i]*dx[:,j]*dx[:,k]).mean(),(dx[:,i]*dx[:,j]*dx[:,k]*dx[:,l]).mean()))

In [ ]:
# ----------- SUM
zsum = np.loadtxt(f'{folder_bin}_histosum_{d}_{label}.dat')
zsums = np.loadtxt(f'{folder_bin}_histosums_{d}_{label}.dat')

In [ ]:
s=[]
for _ in trange(500):
    x, energy_history = thermalize(x,J,h, NumSteps=1000 , crange =range)
    s.append(x.numpy().sum(1))
s=np.array(s).flatten()

In [ ]:
# histo = np.histogram(np.array(x).sum(1), np.arange(-d,d,2)-1, density=True);
histo = np.histogram(s, np.arange(-d,d,2)-1, density=True);
np.savetxt(f'{folder_bin}_histosum_{d}_{label}_MODEL.dat',histo[0])


plt.plot(histo[1][:-1], zsums/zsums.sum()/2, 'o--', label='data shuffled', color='blue')
plt.plot(histo[1][:-1], zsum/zsum.sum()/2, 'o--', label='data', color='black')
plt.plot(histo[1][:-1], histo[0],'o--', label='model', color='red')

plt.title(f'class {label}')
# plt.yscale('log')
# plt.legend()
plt.xlabel(r'$z = \sum_i \sigma_i$')
plt.ylabel('P(z)')

plt.xticks(fontsize=14);
plt.yticks(fontsize=14);

## independent case

In [ ]:
# d=500
# label='all'
# label=2

f_data = np.loadtxt(f'{folder_bin}f_data_{d}_{label}.dat')
q1 = np.log((1-f_data[:d])/(1+f_data[:d]))/2*d
qind = np.hstack((q1, [0]*int(d*(d-1)/2)))

In [ ]:
#---------------------2. Find parameters
h,J = get_hJ(qind,d)
h = tf.convert_to_tensor(h,dtype=tf.float64)
J= tf.convert_to_tensor(J,dtype=tf.float64)

n_samples = 20000
x = np.random.choice([-1,1], (n_samples,d))
x = tf.convert_to_tensor(x,dtype=tf.float64) 

In [ ]:
n_thermalize = 50000
x, energy_history = thermalize(x,J,h, NumSteps=n_thermalize , crange =trange)
plt.plot(energy_history)

In [ ]:
x, f_model = compute_f(x,J,h,StepsTherm=1000,NumSteps=500, crange=lambda NumSteps, dtype: trange(NumSteps))

In [ ]:
np.savetxt(f'{folder_bin}f_model_IND_{d}_{label}.dat',f_model.numpy() )

In [ ]:
m2 = np.loadtxt(f'{folder}_m2_{d}_{label}.dat')[:,0]
m3 = np.loadtxt(f'{folder}_m3_{d}_{label}.dat')[:,0]
m4 = np.loadtxt(f'{folder}_m4_{d}_{label}.dat')[:,0]
indices = np.loadtxt(f'{folder}_m4_{d}_{label}.dat')[:,1:].astype(int)

m2_model=np.zeros(len(m2))
m3_model=np.zeros(len(m2))
m4_model=np.zeros(len(m2))

In [ ]:
ntries=500
for _ in trange(ntries):
    x, energy_history = thermalize(x,J,h, NumSteps=1000 , crange =range)
    dx = x.numpy()-x.numpy().mean(0)
    for ii in range(len(indices)):
        i,j,k,l = indices[ii]   
        m2_model[ii]+= (dx[:, [i,j]].prod(1).mean())
        m3_model[ii]+= (dx[:, [i,j,k]].prod(1).mean())
        m4_model[ii]+= (dx[:, [i,j,k,l]].prod(1).mean())
m2_model/=ntries
m3_model/=ntries
m4_model/=ntries

np.savetxt(f'{folder}_m2IND_{d}_all.dat',m2_model)
np.savetxt(f'{folder}_m3IND_{d}_all.dat',m3_model)
np.savetxt(f'{folder}_m4IND_{d}_all.dat',m4_model)

In [ ]:
np.savetxt(f'{folder_bin}qIND_{d}_{label}.dat', np.array(qind))

In [ ]:
# plt.scatter(m2,m2_model, s=2)
plt.scatter(m3,m3_model, s=2)
plt.scatter(m4,m4_model, s=2)

xmin, xmax = plt.gca().get_xlim()
xx = np.linspace(xmin,xmax,100)
plt.plot(xx,xx, color='red')

## Analysis

In [ ]:
from statistics import mode

In [ ]:
d = 200
label='all'

q = np.loadtxt(f'{folder_bin}q_{d}_{label}.dat')

In [ ]:
zsel = np.loadtxt(f'{folder_bin}_datasel_{d}.dat')[:]
lsel = np.loadtxt(f'{folder_bin}_datasel_label_{d}.dat')[:]

In [ ]:
minima=[]
for i in trange(10000,len(zsel)):
    cell= zsel[i]
    minima.append(get_minimum(cell,q))

In [ ]:
minima= np.array(minima)  
tupled_lst = set(map(tuple, minima))
lst = np.array(list(map(list, tupled_lst)))
basin =[]
for i in range(len(minima)):
    basin.append(np.where((minima[i] == lst).prod(1)==1)[0][0])
basin = np.array(basin)
print(len(lst))

In [ ]:
class_of_basn = np.array([mode(lsel[basin==i]) for i in range(1+int(basin.max()))])
lsel_pred = np.zeros(len(lsel))
for i in range(len(lsel_pred)):
    lsel_pred[i] = class_of_basn[basin[i]]
## complessive accuracy
accuracy = (lsel_pred == lsel).sum()/len(lsel)
print(accuracy)

In [ ]:
# folder = folder_bin = '../../../../scratch/network/csarra/allen_saved/binary/'
np.savetxt(f'{folder}_datasel_labelPRED_{d}.dat',lsel_pred)

In [ ]:
# ------------------------------ SAVE
folder_saved = 'art_saved/'
# acc_ising = []
acc_ising = np.loadtxt(f'{folder_saved}acc_ising.dat')
acc_ising = [i for i in acc_ising]

acc_ising.append((d,accuracy))
np.savetxt(f'{folder_saved}acc_ising.dat', np.array(acc_ising))

In [ ]:
# accuracy per class
acc_per_class = np.array([(lsel_pred[lsel==k] == lsel[lsel==k]).sum()/len(lsel[lsel==k]) for k in [1,2,9,28,29,30,31,33]])

In [ ]:
plt.imshow(lst, aspect='auto', interpolation='None')

In [ ]:
plt.hist(lst.sum(1),200);

In [ ]:
lst_toclass = np.array([mode(lsel[basin==i]) for i in range(len(lst))])

In [ ]:
plt.scatter(lst_toclass, lst.sum(1))

In [ ]:
[(lsel==k).sum() for k in [1,2,9,28,29,30,31,33]]

In [ ]:
plt.hist(lsel_pred[lsel==2], [1,2,9,28,29,30,31,33])

In [ ]:
### ----------- energy barrier
# d = len(x0)

# x = np.copy(x0)
# actual_energy=[]
# h,J = get_hJ(q,d)
# actual_energy.append(get_H(x0[None,:],J,h)[0])

# for _ in range(100000):  #steps allowed to find a minimum  
#     xx = np.repeat(x[None,:],d,0) 

#     num_part,d = xx.shape
#     indices_sel = range(num_part)
#     Dh_h = tf.gather(h, indices_sel)
#     Dh_J = tf.reduce_sum(tf.gather(J, indices_sel, batch_dims=0)*xx,1)
#     DH = -2 * tf.gather(xx,indices_sel, batch_dims=1)*(Dh_h + 2*Dh_J)/d

#     DH = np.array(DH)
#     if DH.min()<0:
#         isel = np.argmin(DH)
#         x[isel]*=-1
#         actual_energy.append(actual_energy[-1]+DH[isel])
#     else:     
#         return x

In [ ]:
# starting from the minima, compute their energy

In [ ]:
h,J = get_hJ(q,d)
energy_lst = get_H(lst,J,h)

In [ ]:
plt.scatter(lst_toclass, energy_lst, s=3)

In [ ]:
plt.plot(energy_lst)

In [ ]:
#start from a certain minimum x0 with energy x0min

lst_barriers =np.zeros(len(lst))
ended_state=[]
for imin in trange(0,len(lst),10):
    x0 = lst[imin]
    x0min = lst[imin]
    d = len(x0)

    x = np.copy(x0)
    actual_energy=[]
    h,J = get_hJ(q,d)
    actual_energy.append(get_H(x0[None,:],J,h)[0])

    for _ in range(5000):
        # find the steepest way up
        xx = np.repeat(x[None,:],d,0) 
        num_part,d = xx.shape
        indices_sel = range(num_part)
        Dh_h = tf.gather(h, indices_sel)
        Dh_J = tf.reduce_sum(tf.gather(J, indices_sel, batch_dims=0)*xx,1)
        DH = -2 * tf.gather(xx,indices_sel, batch_dims=1)*(Dh_h + 2*Dh_J)/d
        DH = np.array(DH)

        if DH.max()>0:
            isel = np.argmax(DH)
            x[isel]*=-1
            actual_energy.append(actual_energy[-1]+DH[isel])

            corr_basin = get_minimum(x,q)

            if (corr_basin==x0).prod()==0:
#                 print('stopped!')

                ended = x
                barrier =(actual_energy[-1]-actual_energy[0]).numpy()
#                 print(ended, barrier)
                lst_barriers[imin] = barrier
                ended_state.append(corr_basin)
                break

In [ ]:
plt.plot(lst_barriers,'o')

In [ ]:
plt.imshow(ended_state,aspect='auto', interpolation='None')

In [ ]:
ended_state=np.array(ended_state)
ended_state_tupled_lst = set(map(tuple, ended_state))
ended_state_lst = np.array(list(map(list, ended_state_tupled_lst)))

In [ ]:
ended_state_lst.shape

In [ ]:
ended_state.shape, lst.shape

In [ ]:
(ended_state[:,None]==lst[None,:]).prod(-1).argmax(1), (ended_state[:,None]==lst[None,:]).prod(-1).max(1)

In [ ]:
get_H(ended_state_lst,J,h)

In [ ]:
### ----------- energy barrier
# for _ in range(100000):  #steps allowed to find a minimum  
#     xx = np.repeat(x[None,:],d,0) 
#     num_part,d = xx.shape
#     indices_sel = range(num_part)
#     Dh_h = tf.gather(h, indices_sel)
#     Dh_J = tf.reduce_sum(tf.gather(J, indices_sel, batch_dims=0)*xx,1)
#     DH = -2 * tf.gather(xx,indices_sel, batch_dims=1)*(Dh_h + 2*Dh_J)/d
#     DH = np.array(DH)
#     if DH.max()>0:
#         isel = np.argmax(DH)
#         x[isel]*=-1
#         actual_energy.append(actual_energy[-1]+DH[isel])       
#         xmin = get_minimum(x) 
#     else:     
        

In [ ]:
np.savetxt(f'{folder_saved}acc_ising.dat', np.array(acc_ising))

In [ ]:
## [per class accuracy]
[(lsel_pred[lsel ==k] == lsel[lsel ==k]).sum()/len(lsel[lsel ==k]) for k in [1,2,9,28,29,30,31,33]]

In [ ]:
h,J = get_hJ(q,d)

In [ ]:
energy_lst = np.array(get_H(lst,J,h))
energy_lst_sorted = np.sort(energy_lst)

lst_sorted = lst[np.argsort(energy_lst)]

In [ ]:
basin_sorted =[]
for i in range(len(minima)):
    basin_sorted.append(np.where((minima[i] == lst_sorted).prod(1)==1)[0][0])
basin_sorted = np.array(basin_sorted)

In [ ]:
basin_sizes = np.array([(basin==i).sum() for i in range(len(lst))])
basin_sorted_sizes = np.array([(basin_sorted==i).sum() for i in range(len(lst))])

In [ ]:
plt.scatter(range(len(lst)), energy_lst_sorted, s=basin_sorted_sizes/20)

In [ ]:
# lst_basin = np.array(list(dict.fromkeys(basin)))

In [ ]:
basin_sorted_sizes

In [ ]:
lst_basin = np.array(list(dict.fromkeys(basin_sorted)))

In [ ]:
# lst_basin

In [ ]:
for i in lst_basin:
    hhisto = plt.hist(lsel[basin==i],  range(34));
    plt.title(f'basin {i}, largest: {hhisto[1][1:][np.argmax(hhisto[0])]}')
    plt.show()

In [ ]:
plt.hist(lsel, range(34))

In [ ]:
label_list = np.array(list(dict.fromkeys(lsel)))

In [ ]:
label_list

In [ ]:
for i in label_list:
    hhisto = plt.hist(basin_sorted[lsel==i],  range(25));
    plt.title(f'class {i}, largest: {hhisto[1][1:][np.argmax(hhisto[0])]}')
    plt.show()

In [ ]:
plt.figure(figsize=(8,5))
for i in range(len(list_classes)):
    plt.subplot(1,2,1)
    plt.scatter(f_data_all[:d], f_data_i[i][:d], s=5)
    plt.xlabel('m_all')
    plt.ylabel('m_class')
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    xmin, xmax = plt.gca().get_xlim()
    xx = np.linspace(xmin,xmax,100)
    plt.plot(xx,xx, color='grey')
    
    plt.subplot(1,2,2)
    plt.scatter(f_data_all[d:], f_data_i[i][d:], s=5)
    plt.xlabel('m2_all')
    plt.ylabel('m2_class')
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    xmin, xmax = plt.gca().get_xlim()
    xx = np.linspace(xmin,xmax,100)
    plt.plot(xx,xx, color='grey')
plt.tight_layout()


In [ ]:
plt.figure(figsize=(8,5))
for i in range(len(list_classes)):
    plt.subplot(1,2,1)
    plt.scatter(q_all[:d]/d, q_i[i][:d]/d, s=15)
    plt.xlabel('h_all')
    plt.ylabel('h_class')
    xmin, xmax = plt.gca().get_xlim()
    xx = np.linspace(xmin,xmax,100)
    plt.plot(xx,xx, color='grey')    
    
    plt.subplot(1,2,2)
    plt.scatter(q_all[d:]/d, q_i[i][d:]/d, s=15)
    plt.xlabel('J_all')
    plt.ylabel('J_class')
    xmin, xmax = plt.gca().get_xlim()
    xx = np.linspace(xmin,xmax,100)
    plt.plot(xx,xx, color='grey')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
for i in range(len(list_classes)):
    histo = np.histogram(q_i[i][:d]/d,density=True)
    plt.plot(histo[1][1:], histo[0])
    plt.xlabel('h')
    plt.ylabel('P(h)')
    
plt.subplot(1,2,2)

for i in range(len(list_classes)):
    histo = np.histogram(q_i[i][d:]/d,100,density=True)
    plt.plot(histo[1][1:], histo[0])
    plt.xlabel('J')
    plt.ylabel('P(J)')
plt.xlim(-0.2,0.2)

plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
for i in range(len(list_classes)):
    plt.subplot(1,2,1)
    histo = np.histogram(f_data_i[i][:d], 10, density=True)
    plt.plot(histo[1][1:], histo[0])
    
    plt.subplot(1,2,2)
    histo = np.histogram(f_data_i[i][d:], 100, density=True)
    plt.plot(histo[1][1:], histo[0])
plt.tight_layout()

In [ ]:
NumSteps=1000
StepsTherm=10
d = len(x[0])
energy_x_class =[]
for i in range(N_CLASSES):
    q = q_i[i]
    h,J = get_hJ(q,d)
    H1 = get_H(x,J,h)
    energy_x_class.append(H1)
energy_x_class = np.array(energy_x_class)

h2,J2 = get_hJ(q_all,d)
h2 = tf.convert_to_tensor(h2,dtype=tf.float64)
J2= tf.convert_to_tensor(J2,dtype=tf.float64)
H2 = get_H(x,J2,h2)

num_part,d = x.shape
num_par = int(d+ d*(d-1)/2)

ratio = np.zeros(N_CLASSES)
ratio_history=[]
for n in trange(NumSteps):
    for _ in tf.range(StepsTherm):
        x,H2 = one_step_MC(x, h2,J2,H2)
        
    energy_x_class =[]
    for i in range(N_CLASSES):
        q = q_i[i]
        h,J = get_hJ(q,d)
        H1 = get_H(x,J,h)
        energy_x_class.append(H1)
    energy_x_class = np.array(energy_x_class) 
    

    ratio_partial = tf.reduce_mean(tf.exp(-(energy_x_class-H2)),1)
    ratio = (n*ratio + ratio_partial)/(n+1)  
    ratio_history.append(np.array(ratio).mean())
ratio = ratio.numpy()

In [ ]:
ratio = np.zeros(N_CLASSES)
ratio_history=[]
n=0

In [ ]:
NumSteps=10000
for _ in trange(NumSteps):
    for _ in tf.range(StepsTherm):
        x,H2 = one_step_MC(x, h2,J2,H2)
        
    energy_x_class =[]
    for i in range(N_CLASSES):
        q = q_i[i]
        h,J = get_hJ(q,d)
        H1 = get_H(x,J,h)
        energy_x_class.append(H1)
    energy_x_class = np.array(energy_x_class) 
    

    ratio_partial = tf.reduce_mean(tf.exp(-(energy_x_class-H2)),1)
    ratio = (n*ratio + ratio_partial)/(n+1)  
    ratio_history.append(np.array(ratio).mean())
    n+=1
ratio = ratio.numpy()

In [ ]:
plt.plot(ratio_history[:])

In [ ]:
ratio

## ~ bayes 

In [ ]:
# load Pclass from labels
pclass_ = np.loadtxt(f'{folder_bin}pclass.dat')
if (pclass_[:,0].astype(int) == list_classes).prod() ==1:
    pclass = pclass_[:,1]
    print('pclass correctly loaded')
else: 
    print('there is a problem')
    print(pclass_[:,0].astype(int), list_classes)

In [ ]:
# load x,y to test the classifier
x = np.loadtxt(f'{folder_bin}_datasel_{d}.dat')[:5000]
y = np.loadtxt(f'{folder_bin}_datasel_label_{d}.dat')[:5000]

In [ ]:
# find H(x|y)

energy_x_class =[]

for i in trange(N_CLASSES):
    q = q_i[i]
    h,J = get_hJ(q,d)
    
    H1 = get_H(x,J,h)
    energy_x_class.append(H1)
energy_x_class = np.array(energy_x_class)

In [ ]:
# find P(y|x) propto exp(-H_class)P_class/r_class
prob_y_given_x_nn = np.exp(-energy_x_class)*pclass[:,None]/ratio[:,None]
prob_y_given_x = prob_y_given_x_nn/prob_y_given_x_nn.sum(0)
np.savetxt(f'{folder_bin}pyx_{d}.txt',prob_y_given_x)

## load p(y|x)

In [ ]:
prob_y_given_x = np.loadtxt(f'{folder_bin}pyx_{d}.txt')

In [ ]:
cell = 100
plt.bar(np.arange(N_CLASSES), prob_y_given_x[:,cell])

In [ ]:
plt.hist(prob_y_given_x.flatten(),100);
plt.yscale('log')

In [ ]:
y_pred_temp = np.argmax(prob_y_given_x,0)
y_pred = np.copy(y_pred_temp)
for i in range(N_CLASSES):
    y_pred[y_pred_temp==i]=list_classes[i]

In [ ]:
acc=(y_pred == y).sum()/len(y)

In [ ]:
folder_saved = 'art_saved/'

In [ ]:
# acc_isingMIx = np.loadtxt(f'{folder_saved}acc_isingMix.dat')
# ###-----------------------------------
# # acc_isingMIx =[] #-------- only if there isn't acc_n
# acc_isingMIx = [i for i in acc_isingMIx]

In [ ]:
# acc_isingMIx.append((d,acc))

In [ ]:
# acc_isingMIx.append((d, acc))    
# np.savetxt(f'{folder_saved}acc_isingMix.dat', np.array(acc_isingMIx))

In [ ]:
plt.hist(y, N_CLASSES*10,alpha=0.5);
plt.hist(y_pred, N_CLASSES*10,alpha=0.5);

# LOAD CLASSIFIER DATA

In [ ]:
N_CLASSES=8

In [ ]:
# load Pclass from labels
pclass_ = np.loadtxt(f'{folder_bin}pclass.dat')
pclass = pclass_[:,1]
list_classes =np.array([1,30,31,33,9,29,2,28])
H_y = -(pclass*np.log(pclass)).sum()

In [ ]:
#__________________________________________________________________________________
# ---------------------- Conditional independent model
#__________________________________________________________________________________

cond_ind_classifier =[]
cond_ind_mi =[]
for d in [10,15,50,100,200,300,400,500]:
    x = np.loadtxt(f'{folder_bin}_datasel_{d}.dat')
    y = np.loadtxt(f'{folder_bin}_datasel_label_{d}.dat')
    list_classes =np.array([1,30,31,33,9,29,2,28])
    label='all'
    prob_y_given_x_nn=[]
    ii=0
    for label in list_classes:

        f_data = np.loadtxt(f'{folder_bin}f_data_{d}_{label}.dat')
        mu = f_data[:d]
        a_ind_real = 1/2*np.log((1-mu+1e-8)/(1+mu))
        prob_x_given_y = (np.exp(-a_ind_real*x)/(np.exp(a_ind_real)+np.exp(-a_ind_real))).prod(-1)

        prob_y_given_x_nn.append(prob_x_given_y*pclass[ii])

        ii+=1
    prob_y_given_x_nn = np.vstack((prob_y_given_x_nn))

    prob_y_given_x = prob_y_given_x_nn/prob_y_given_x_nn.sum(0)
    y_pred_temp = np.argmax(prob_y_given_x, 0)

    y_pred = np.copy(y_pred_temp)
    for i in range(N_CLASSES):
        y_pred[y_pred_temp==i]=list_classes[i]
    cond_ind_classifier.append((d,(y_pred == y).sum()/len(y)))
    
    
    H_y_given_x = (-(prob_y_given_x*np.log(prob_y_given_x)).sum(0)).mean()
    mutual_info = H_y - H_y_given_x
    cond_ind_mi.append((d, mutual_info))
cond_ind_mi = np.array(cond_ind_mi)

In [ ]:
class_accuracy=[]
for d in [10,50,100]:
    prob_y_given_x = np.loadtxt(f'{folder_bin}pyx_{d}.txt')
    y = np.loadtxt(f'{folder_bin}_datasel_label_{d}.dat')    
    plt.hist(prob_y_given_x.flatten(),100, label=d);
    
    y_pred_temp = np.argmax(prob_y_given_x,0)
    y_pred = np.copy(y_pred_temp)
    for i in range(N_CLASSES):
        y_pred[y_pred_temp==i]=list_classes[i]
    class_accuracy.append((d,(y_pred == y).sum()/len(y)))
    
    
plt.yscale('log')
plt.legend()

In [ ]:
plt.axhline(H_y, color='grey', ls='--')
plt.ylabel('I(x,class)')
plt.xlabel('n genes')


dmi =[]
for d in [10,15,50,100]:
    prob_y_given_x = np.loadtxt(f'{folder_bin}pyx_{d}.txt')
    # we want this to be as small as possible! (max value is H_y)
    H_y_given_x = (-(prob_y_given_x*np.log(prob_y_given_x)).sum(0)).mean()
    mutual_info = H_y - H_y_given_x
    plt.scatter(d, mutual_info, color='black')
    dmi.append((d,mutual_info))
dmi = np.array(dmi)
    
# plt.scatter(cond_ind_mi[:,0], cond_ind_mi[:,1])

In [ ]:
# class_accuracy = np.array(class_accuracy)
# plt.scatter(class_accuracy[:,0], class_accuracy[:,1])

#------------------------ NN classifier
saved_classifier = np.loadtxt(f'{folder_bin}Classifier.txt')
plt.scatter(saved_classifier[:,0][-1], saved_classifier[:,1][-1], color='red')
nn = np.array(list(dict.fromkeys(saved_classifier[:,0])))
class_mean = np.array([saved_classifier[:,1][saved_classifier[:,0]==nn[i]].mean() for i in range(len(nn))])
class_std  = np.array([saved_classifier[:,1][saved_classifier[:,0]==nn[i]].std() for i in range(len(nn))])
class_num = np.array([(saved_classifier[:,0]==nn[i]).sum() for i in range(len(nn))])
plt.errorbar(nn, class_mean, class_std/np.sqrt(class_num-1), ls='--', color='red')
plt.scatter(nn, class_mean, color='red', label='classifier')

# ----------------------- ISING MODEL
class_accuracy = np.array(class_accuracy)
plt.scatter(class_accuracy[:,0], class_accuracy[:,1], color='black', label='Ising mixture')

# #------------------------- independent
# cond_ind_classifier = np.array(cond_ind_classifier)
# plt.scatter(cond_ind_classifier[:,0], cond_ind_classifier[:,1], label =' cond ind')

plt.legend()

# plt.yscale('log')
# plt.xscale('log')

plt.xlabel('number of genes')
plt.ylabel('accuracy')

# Heat capacity

In [ ]:
d = 100

for label in [2]:
    q = np.loadtxt(f'{folder_bin}q_{d}_{label}.dat')
    h,J=get_hJ(q,d)
    
    # thermalize particles at T=1
    n_samples = 1000
    x = np.random.choice([-1,1], (n_samples,d))
    x = tf.convert_to_tensor(x,dtype=tf.float64) 
    
    for _ in range(2):
        n_thermalize = 10000
        x, energy_history = thermalize(x,J,h, NumSteps=n_thermalize , crange =trange)
        plt.plot(energy_history)
        plt.show()
    x1=np.copy(x)
    
    
    heat_capacity =[]
    # ---------------------
    e_mean = (np.array(get_H(x,J,h))).mean()
    e_var = ((np.array(get_H(x,J,h)) - e_mean)**2).mean()
    n_thermalize = 100
    x=np.copy(x1)
    for T in np.linspace(1,3,20):
        e_var =0
        x, energy_history = thermalize(x,J/T,h/T, NumSteps=n_thermalize , crange =trange)
        for k in range(1,3):
            x, energy_history = thermalize(x,J/T,h/T, NumSteps=50 , crange =range)
            e_mean = (np.array(get_H(x,J,h))).mean()
            e_var1 = ((np.array(get_H(x,J,h)) - e_mean)**2).mean()
            e_var = (e_var*(k-1) + e_var1)/k
        heat_capacity.append((T,e_var/T**2))
    x=np.copy(x1)
    n_thermalize = 5000
    for T in np.linspace(0.95,0.1,10):
        e_var =0
        x, energy_history = thermalize(x,J/T,h/T, NumSteps=n_thermalize , crange =trange)
        for k in range(1,3):
            x, energy_history = thermalize(x,J/T,h/T, NumSteps=200 , crange =range)
            e_mean = (np.array(get_H(x,J,h))).mean()
            e_var1 = ((np.array(get_H(x,J,h)) - e_mean)**2).mean()
            e_var = (e_var*(k-1) + e_var1)/k
        heat_capacity.append((T,e_var/T**2)) 
        
    hc = np.array(heat_capacity)[:-1]
    plt.scatter(hc[:,0], hc[:,1])
    plt.show()
    np.savetxt(f'hc_{d}_{label}.dat', hc)

In [ ]:
plt.figure(figsize=(10,5))

d= 10
plt.subplot(1,2,1)    
for label in [2,9,28,29,30,31,33]:
    hc = np.loadtxt(f'hc_{d}_{label}.dat')
    plt.plot(hc[:,0][np.argsort(hc[:,0])], hc[:,1][np.argsort(hc[:,0])]/d)
plt.gca().set_prop_cycle(None)    
plt.xlabel('T')
plt.ylabel('C(T)/N')
plt.axvline(1,ls='--', color='grey')
plt.ylim(0,1.5)
plt.title('N=10')


d= 100
plt.subplot(1,2,2)    
for label in [2,9,28,29,30,31,33]:
    hc = np.loadtxt(f'hc_{d}_{label}.dat')
    plt.plot(hc[:,0][np.argsort(hc[:,0])], hc[:,1][np.argsort(hc[:,0])]/d)
plt.gca().set_prop_cycle(None)    
plt.xlabel('T')
plt.ylabel('C(T)/N')
plt.axvline(1,ls='--', color='grey')
plt.ylim(0,1.5)
plt.title('N=100')

plt.tight_layout()

In [ ]:
# hc_all = np.copy(hc)
# hc_33 = np.copy(hc)
hc_9= np.copy(hc)

In [ ]:
for hc in [hc_9]:
    plt.plot(hc[:,0][np.argsort(hc[:,0])], hc[:,1][np.argsort(hc[:,0])])
    
# hc   = hc_all
# plt.plot(hc[:,0][np.argsort(hc[:,0])], hc[:,1][np.argsort(hc[:,0])], color='black', lw=2)

plt.axvline(1,ls='--', color='grey')

In [ ]:
# np.savetxt(f'{folder_bin}heat_capacity_{d}.txt',hc)

In [ ]:
# heat_capacity =[]
# for T in tqdm(np.hstack((np.linspace(0.1,1,10), np.linspace(1.1,3,10)))):
#     prob = np.exp(-a_ind_real/T)/2/np.cosh(a_ind_real/T)  
#     en =[]
#     for _ in range(100):
#         x = np.array([np.random.choice([1,-1],p=(prob[i], 1-prob[i]),  size=500) for i in range(len(prob))]).T
#         en.append(np.array(get_H(x,J,h)))
#     en=np.array(en)
#     e_var = np.var(en)
#     heat_capacity.append((T, e_var/T**2))

In [ ]:
d=10

all_points = get_p(d)
mixed = ((all_points[:,None,:]*all_points[:,:,None])[:,np.triu_indices(d, k=1)[0],np.triu_indices(d, k=1)[1]])
v_points = np.hstack((all_points,mixed))

In [ ]:
for label in [1,2,9,28,29,30,31,33]:
    q = np.loadtxt(f'{folder_bin}q_{d}_{label}.dat')
    h,J=get_hJ(q,d)
    hc =[]
    for T in np.linspace(0.1,5,100):
        h =np.array(tf.reduce_sum(v_points*q,1)/d)
        p = np.exp(-h/T); p/=p.sum()
        mean_energy = (h*p).sum() 
    #     var = ((h -mean_energy)**2*p).sum() 
        var = (h**2*p).sum() - (h*p).sum()**2 
        hc.append((T,var/T**2))
    hc = np.array(hc)
    np.savetxt(f'hc_{d}_{label}.dat', hc)
    plt.plot(hc[:,0], hc[:,1])
plt.xlabel('T')
plt.ylabel('C(T)')

label='all'
q = np.loadtxt(f'{folder_bin}q_{d}_{label}.dat')
h,J=get_hJ(q,d)
hc =[]
for T in np.linspace(0.1,5,100):
    h =np.array(tf.reduce_sum(v_points*q,1)/d)
    p = np.exp(-h/T); p/=p.sum()
    mean_energy = (h*p).sum() 
#     var = ((h -mean_energy)**2*p).sum() 
    var = (h**2*p).sum() - (h*p).sum()**2 
    hc.append((T,var/T**2))
hc = np.array(hc)
plt.plot(hc[:,0], hc[:,1],color='black' ,lw=2)
plt.xlabel('T')
plt.ylabel('C(T)')

plt.axvline(1,ls='--', color='grey')

# Conditional independent model

In [ ]:
from tqdm import tqdm 

In [ ]:
d = 200
label='all'
f_data = np.loadtxt(f'{folder_bin}f_data_{d}_{label}.dat')
mu = f_data[:d]
a_ind_real = np.log((1-mu+1e-10)/(1+mu))/2
q = np.hstack((a_ind_real*d, [0]*int(d*(d-1)/2)))
h,J=get_hJ(q,d)

In [ ]:
heat_capacity =[]

In [ ]:
for T in tqdm(np.hstack((np.linspace(0.1,1,10), np.linspace(1.1,3,10)))):
    prob = np.exp(-a_ind_real/T)/2/np.cosh(a_ind_real/T)  
    en =[]
    for _ in range(100):
        x = np.array([np.random.choice([1,-1],p=(prob[i], 1-prob[i]),  size=500) for i in range(len(prob))]).T
        en.append(np.array(get_H(x,J,h)))
    en=np.array(en)
    e_var = np.var(en)
    heat_capacity.append((T, e_var/T**2))

In [ ]:
hc = np.array(heat_capacity)
plt.scatter(hc[:,0],hc[:,1])


In [ ]:
np.savetxt(f'{folder_bin}heat_capacity_IND_{d}.txt',hc)

# plot hc

In [ ]:
for d in [50,100,200]:
    hc = np.loadtxt(f'{folder_bin}heat_capacity_{d}.txt')  
    plt.scatter(hc[:,0], hc[:,1]/d, label= d)
    plt.plot(np.sort(hc[:,0]), hc[:,1][np.argsort(hc[:,0])]/d, '--')
    
    
plt.axvline(1, ls='--', color='grey')
plt.legend()

plt.xlim(0,3)
plt.ylim(0,1)

plt.xlabel('T')
plt.ylabel('C(T)/N')
plt.title('Ising model')

plt.xticks(fontsize=14);
plt.yticks(fontsize=14);

In [ ]:
for d in [50,100,200]:
    hc = np.loadtxt(f'{folder_bin}heat_capacity_IND_{d}.txt')  
    plt.scatter(hc[:,0], hc[:,1]/d, label= d)
    plt.plot(np.sort(hc[:,0]), hc[:,1][np.argsort(hc[:,0])]/d, '--')
plt.axvline(1, ls='--', color='grey')


plt.legend()
plt.xlabel('T')
plt.ylabel('C(T)/N')
plt.xlim(0,3)
plt.ylim(0,1)
plt.title('Independent model')

plt.xticks(fontsize=14);
plt.yticks(fontsize=14);

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(1,2,1)

for d in [50,100,200]:
    hc = np.loadtxt(f'{folder_bin}heat_capacity_{d}.txt')  
    plt.scatter(hc[:,0], hc[:,1]/d, label= d)
    plt.plot(np.sort(hc[:,0]), hc[:,1][np.argsort(hc[:,0])]/d, '--')
    
    
plt.axvline(1, ls='--', color='grey')
plt.legend()

plt.xlim(0,3)
plt.ylim(0,1)

plt.xlabel('T')
plt.ylabel('C(T)/N')
plt.title('Ising model')

plt.xticks(fontsize=14);
plt.yticks(fontsize=14);

plt.subplot(1,2,2)
for d in [50,100,200]:
    hc = np.loadtxt(f'{folder_bin}heat_capacity_IND_{d}.txt')  
    plt.scatter(hc[:,0], hc[:,1]/d, label= d)
    plt.plot(np.sort(hc[:,0]), hc[:,1][np.argsort(hc[:,0])]/d, '--')
plt.axvline(1, ls='--', color='grey')


plt.legend()
plt.xlabel('T')
plt.ylabel('C(T)/N')
plt.xlim(0,3)
plt.ylim(0,1)
plt.title('Independent model')

plt.xticks(fontsize=14);
plt.yticks(fontsize=14);


plt.tight_layout()

In [ ]:
zsel = np.loadtxt(f'{folder}_datasel_{d}.dat')[:100]
lsel = np.loadtxt(f'{folder}_datasel_label_{d}.dat')[:100]

In [ ]:
q = np.loadtxt(f'{folder_bin}q_{d}_{label}.dat')
q[:d]=np.zeros(len(q[:d]))

minima=[]
for i in trange(len(zsel)):
    cell= zsel[i]
    minima.append(get_minimum(cell,q))

In [ ]:
minima= np.array(minima)  
tupled_lst = set(map(tuple, minima))
lst = np.array(list(map(list, tupled_lst)))
basin =[]
for i in range(len(minima)):
    basin.append(np.where((minima[i] == lst).prod(1)==1)[0][0])
basin = np.array(basin)
print(len(lst))

In [ ]:
basin

In [ ]:
class_of_basn = np.array([mode(lsel[basin==i]) for i in range(1+int(basin.max()))])
lsel_pred = np.zeros(len(lsel))
for i in range(len(lsel_pred)):
    lsel_pred[i] = class_of_basn[basin[i]]
    
## complessive accuracy
accuracy = (lsel_pred == lsel).sum()/len(lsel)
print(accuracy)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
zsel = np.loadtxt(f'{folder}_datasel_{d}.dat')[:1000]
lsel = np.loadtxt(f'{folder}_datasel_label_{d}.dat')[:1000]

pca = PCA(n_components=2)
pca.fit(zsel)

In [ ]:
zsel_pca = pca.transform(zsel)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
for i in list_classes:
    plt.scatter(zsel_pca[lsel==i][:,0], zsel_pca[lsel==i][:,1], s=5, label=i)
plt.legend(bbox_to_anchor=(1,1))